<a href="https://colab.research.google.com/github/sportcman/Gpt/blob/main/%D0%9C%D0%B8%D0%BD%D0%B8_Gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
pip install transformers[torch]

In [ ]:
pip install torch_xla

In [ ]:
pip install accelerate -U

In [ ]:
pip install torch torchvision torchaudio

In [ ]:
pip install tqdm

**`Создание модели. Mini.`**

In [ ]:
from transformers import GPT2Tokenizer, GPT2Config, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('/gdrive/MyDrive/TrenerGpt/tokenizer')

model_config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_layer=8, # Количество слоев (layers) в трансформере. Большее количество слоев может помочь модели улавливать более сложные зависимости в данных, но требует больше вычислительных ресурсов
    n_head=16, # Количество "голов" (heads) в механизме внимания (attention mechanism). Большее количество голов позволяет модели фокусироваться на разных аспектах данных одновременно и улучшает ее способность к обучению зависимостей
    n_embd=1024, # Размерность вектора эмбеддинга для каждого токена. Определяет сколько информации может содержаться в представлении каждого слова
    intermediate_size=2048, # Размер скрытого слоя промежуточного представления. Определяет размерность скрытого слоя между слоем внимания и слоем линейного преобразования
    hidden_size=1024, # Размер скрытого состояния модели. Определяет размерность скрытого состояния каждого слоя трансформера
    max_position_embeddings=2048, # Максимальная длина последовательности, которую модель может обрабатывать. В итоге и выводить не более задоной цыфры. Если последовательность превышает это значение, то она будет обрезана или заполнена специальными токенами
    gradient_checkpointing=True, # Флаг, указывающий, используется ли чекпоинтинг градиентов для оптимизации памяти и вычислений во время обучения
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,    # False=Лож
    pad_token_id=tokenizer.pad_token_id,    # True=Истина
    sep_token_id=tokenizer.sep_token_id,
    use_cache=True, # Флаг, указывающий, используется ли кэширование в процессе генерации текста для ускорения работы модели
    layer_norm_epsilon=1e-5, # Эпсилон для Layer normalization, используемый для стабилизации обучения
    initializer_range=0.01, # Диапазон инициализации весов модели. Определяет начальные значения весов при инициализации модели
    output_attentions=True, # output_attentions, output_hidden_states: Флаги, указывающие, нужно ли выводить внимание и скрытые состояния модели во время прямого прохода
    output_hidden_states=True,
    tie_word_embeddings=True # Флаг, указывающий, нужно ли связывать веса слов в эмбеддингах с весами выходного слоя
)
model = GPT2LMHeadModel(config=model_config)
model.set_input_embeddings(model.resize_token_embeddings(len(tokenizer)))

model.save_pretrained('/gdrive/MyDrive/TrenerGptmini/model')
tokenizer.save_pretrained('/gdrive/MyDrive/TrenerGptmini/model')


Обучение МОДЕЛИ.

In [ ]:
import argparse
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

class CustomTextDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size):
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

        self.examples = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size, truncation=True, padding="max_length")["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i], dtype=torch.long)

def train(model, dataset, tokenizer, device, batch_size, epochs, model_path):
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    scheduler = StepLR(optimizer, step_size=1, gamma=0.95)

    model.train()
    model.to(device)

    for epoch in range(epochs):
        epoch_progress = tqdm(data_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for batch in epoch_progress:
            batch = batch.to(device)
            inputs, labels = batch, batch
            optimizer.zero_grad()
            outputs = model(inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            epoch_progress.set_postfix(loss=loss.item())

        scheduler.step()
        print(f"Epoch {epoch+1}/{epochs} completed.")

    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_epochs', type=int, default=1) # коллличество эпох
    parser.add_argument('--dataset_path', type=str, default="/gdrive/MyDrive/TrenerGptmini/dataset.txt")
    parser.add_argument('--device', type=str, default="cuda" if torch.cuda.is_available() else "cpu")

    args = parser.parse_known_args()[0]

    dataset_path = args.dataset_path
    model_path = "/gdrive/MyDrive/TrenerGptmini/model"
    num_epochs = args.num_epochs
    batch_size = 2 #  должен учитывать количество ядер процессора и доступную оперативную память

    device = torch.device(args.device)

    try:
        tokenizer = GPT2Tokenizer.from_pretrained(model_path)
        model = GPT2LMHeadModel.from_pretrained(model_path)
    except Exception as e:
        print(f"Error loading model from {model_path}, loading GPT-2 base model instead. Error: {e}")
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        model = GPT2LMHeadModel.from_pretrained('gpt2')

    model.to(device)

    dataset = CustomTextDataset(tokenizer, dataset_path, block_size=2048) #  Учитывает среднюю длину текстовых фрагментов, которые модель будет обрабатывать.

    train(model, dataset, tokenizer, device, batch_size, num_epochs, model_path)

if __name__ == '__main__':
    main()


***Скрипт для проверки ответов.***

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

class GPT2Generator:
    def __init__(self, model_path):
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_path)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = GPT2LMHeadModel.from_pretrained(model_path).to(self.device)

    def generate_text(self, input_text, temperature_value, length_value, num_results, no_repeat_ngram_size):
        input_ids = self.tokenizer.encode(input_text, return_tensors='pt').to(self.device)
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)

        outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=length_value,
            num_return_sequences=num_results,
            no_repeat_ngram_size=no_repeat_ngram_size,
            repetition_penalty=1.5,
            temperature=temperature_value,
            do_sample=True
        )

        result_text = ""
        for i, output in enumerate(outputs):
            generated_text = self.tokenizer.decode(output, skip_special_tokens=True)
            result_text += f"Результат {i+1}:\n{generated_text}\n\n"

        return result_text

gpt2_generator = GPT2Generator("/gdrive/MyDrive/TrenerGptmini/model")
temperature_value = 0.1
length_value = 100
num_results = 1
ngram_value = 2

def generate_text():
    input_text = input("Введи затравку: ")
    result_text = gpt2_generator.generate_text(input_text, temperature_value, length_value, num_results, ngram_value)
    print(result_text)


if __name__ == "__main__":
    while True:
        user_input = input("Выберите действие (1 - сгенерировать текст, 2 - выход): ")
        if user_input == "1":
            generate_text()
        elif user_input == "2":
            break
        else:
            print("Некорректный ввод. Попробуйте снова.")
